The code below analyses user interactions and clicks on an app

Dataset:
https://www.dropbox.com/s/4vkv2m12s0bzr9e/test_sql_dataset.zip?dl=0


**Subject:**
Given the provided event dataset that contains the columns:
“created_at”: timestamp of the application opening by a user
“user_created_at”: creation date of the user
“id”: id of the user

In [2]:
import pandas as pd  
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns

import requests
import json
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth #authentication code
from getpass import getpass
from datetime import datetime

import time

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [31]:
import os

#get all names of json files
directory = 'test_sql_dataset/'
json_files = [pos_json for pos_json in os.listdir(directory) if pos_json.endswith('.json')]

#for loop to open json, save into df and append to the main df
data = pd.DataFrame([])
for file in json_files:
    df = pd.read_json(directory +file, lines=True)
    data = data.append(df)

In [34]:
#create index 
data['index'] = np.arange(len(data))
data.set_index('index', inplace = True)

In [35]:
data

,created_at,user_created_at,id
index,,,
0,2018-07-27 09:51:24.841000+00:00,2018-06-22 11:59:20.965000+00:00,2777a0a79861c04f8437f255bce0403959d17e3af44dad...
1,2018-07-27 01:17:22.203000+00:00,2018-06-22 11:59:20.965000+00:00,2777a0a79861c04f8437f255bce0403959d17e3af44dad...
2,2018-07-01 03:05:19.595000+00:00,2018-06-22 11:59:20.965000+00:00,2777a0a79861c04f8437f255bce0403959d17e3af44dad...
3,2018-07-20 07:12:07.705000+00:00,2018-06-22 11:59:20.965000+00:00,2777a0a79861c04f8437f255bce0403959d17e3af44dad...
4,2018-07-20 14:29:34.228000+00:00,2018-06-22 11:59:20.965000+00:00,2777a0a79861c04f8437f255bce0403959d17e3af44dad...
...,...,...,...
513755,2018-08-22 11:34:48.174000+00:00,2018-08-21 19:09:05.250000+00:00,22d24aa964129845ec7c4e410c139fb5f955e5cfbedee4...
513756,2018-08-22 07:32:20.230000+00:00,2018-08-21 19:09:05.250000+00:00,22d24aa964129845ec7c4e410c139fb5f955e5cfbedee4...
513757,2018-08-21 19:09:05.710000+00:00,2018-08-21 19:09:05.250000+00:00,22d24aa964129845ec7c4e410c139fb5f955e5cfbedee4...


In [36]:
data.dtypes

created_at         datetime64[ns, UTC]
user_created_at    datetime64[ns, UTC]
id                              object
dtype: object

In [37]:
data['created_at'] = pd.to_datetime(data['created_at'])
data['user_created_at'] = pd.to_datetime(data['user_created_at'])

In [38]:
data.groupby(['id'])['created_at'].agg(['count']).sort_values(by='count', ascending=False)

,count
id,
68627a5d899a8fd78006db4b5b11c3d23e1819b34a441db2b5c77473df752ba2,4823
c88170fc8971912fd41ba9c793535a18757247e0179f4682f7f6191a79d18d9c,3902
5014bfffaec421c92bf9ebcfcced5d57e6ca4bddb71713c92747beece2a0e788,3616
18f94aefbfc948861262c6a4d4248aad80c9cb85ef93e38446b5ed9ba7f302f9,3285
1298c49b4db0a40e3dcd317f6809f712ee7aeabbf573f3c2b37fd35c1c5bee4a,2930
...,...
42606cefe4e042c60dec89c8dc3878226e0bffba189d2cc6e4724d4612125313,1
10b376fc52c37cf79878b88d145fd67d9bb7426b7139694983e5cc672059a984,1
efd03fa9f5f80c4effe696c1cf70f0c343b6b4cd85149141296a0b2121c2722b,1


In [39]:
data['seniority'] = (data['created_at'] - data['user_created_at']).dt.days

In [40]:
data['user_created_at'] = data['user_created_at'].dt.strftime('%Y-%m-%d')

In [41]:
#we only want to check the 2nd week time frame 
data_new = data[(data['seniority']<14) & (data['seniority']>7)]
data_new['created_at'] = data_new['created_at'].dt.strftime('%Y-%m-%d')
data_new

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,created_at,user_created_at,id,seniority
index,,,,
2,2018-07-01,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,8
7,2018-07-02,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,9
8,2018-07-02,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,10
10,2018-07-01,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,9
11,2018-07-01,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,8
...,...,...,...,...
513578,2018-07-17,2018-07-04,4c466d6e7d16f7418172e2ba3808d252348414c07fe6b9...,12
513579,2018-07-16,2018-07-04,4c466d6e7d16f7418172e2ba3808d252348414c07fe6b9...,11
513580,2018-07-16,2018-07-04,4c466d6e7d16f7418172e2ba3808d252348414c07fe6b9...,11


In [42]:
#count distinct ids that used more than 3days the app during their 2nd week 
final = data_new.groupby('id')['created_at'].nunique().to_frame()
final

,created_at
id,
006f8c08ebbec52f2f59c71e7dfe2bfed30c6ca3275dabe7ad38735e8ca27562,6
00c2a4cfaa8550adc5883da37a6ad90e0162b7d62ed8df13577a3e354126e8d3,6
00def8665d77d215481fae58fc5b3ec820cf040ab2b5e84c167f31238e9d3815,5
014aaf1352e395a0cf24c810b3062694c10ee5adc87bb482be9c69f16e2c32ae,1
015461eade73cc7eea6393f90f96ff581f48a648808b66d834e86b2943337fc8,1
...,...
fdf867573b52df0ff0e468b33b5b2d42897d7d126c86da4a70fa2fbdc374b0af,3
fe8d10187851fc16390c03a392629061a11748d7d1d6032807ce6624ebc6c7d7,6
ff301e8d52a4491c096ec3f15c43fa9822df5d465a9e844a675f9a7fd5cbd2d0,6


In [53]:
proportion = final[final['created_at']>2].shape[0]/final.shape[0]
proportion
print("The proportion of the users created at a given day who open the application at least 3 distinct days, within the second week after the user creation, is",
     round(proportion,1)*100, "%")
      

The proportion of the users created at a given day who open the application at least 3 distinct days, within the second week after the user creation, is 60.0 %


**BONUS QUESTION**

What it is the number of distinct days (which was 3 in the exercise) that best
correlates with the 30 days retention?

In [44]:
#let's see the dataset where seniority is around 30 days (I'll give it a 5 day error window)
data_30 = data[(data['seniority']<35) &(data['seniority']>25)]
data_30['created_at'] = data_30['created_at'].dt.strftime('%Y-%m-%d')

data_30

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,created_at,user_created_at,id,seniority
index,,,,
0,2018-07-27,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,34
1,2018-07-27,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,34
3,2018-07-20,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,27
4,2018-07-20,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,28
5,2018-07-20,2018-06-22,2777a0a79861c04f8437f255bce0403959d17e3af44dad...,27
...,...,...,...,...
513585,2018-08-02,2018-07-04,4c466d6e7d16f7418172e2ba3808d252348414c07fe6b9...,29
513586,2018-08-02,2018-07-04,4c466d6e7d16f7418172e2ba3808d252348414c07fe6b9...,28
513742,2018-08-31,2018-07-27,d102b62060474fc6979f8baff112db9a7d617b260dda6f...,34


In [52]:
#how many distinct days have costumers with 30 days seniority 
final_30 = data_30.groupby('id')['created_at'].nunique().to_frame().sort_values(by=['created_at'], ascending= False)
round(final_30['created_at'].mean(),0)

5.0

On average they have been using 5 distinct days since joined the app. Given this, I believe a 5 day distinct usage best correlates with a 30 day retention.